In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LinearRegression
from sklearn import preprocessing
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import SelectFromModel
from sklearn.cluster import DBSCAN
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score
from sklearn.linear_model import Ridge
from sklearn.svm import SVC
from sklearn.metrics import balanced_accuracy_score
from sklearn.ensemble import BaggingClassifier
import xgboost as xgb
from imblearn.over_sampling import SMOTE

In [7]:
xtrain = pd.read_csv("X_train.csv")
xtrain = xtrain.iloc[:, 1:]
xtest = pd.read_csv("X_test.csv")
xtest = xtest.iloc[:, 1:]
ytrain = pd.read_csv("Y_train.csv")

# Remove id from ytrain
ytrain = ytrain.drop('id', axis = 1)
ytrain.head()

,y
0,1
1,0
2,1
3,1
4,1


In [8]:
sm = SMOTE(random_state=42)
X_res, y_res = sm.fit_resample(xtrain.values, ytrain.values)

xgabb = pd.DataFrame(data = {: bootstrap_y.values}, index = bootstrap_y.index)

SyntaxError: invalid syntax (<ipython-input-8-cb5a46e17e92>, line 4)

In [ ]:
def generate_bootstrap_sample(x_df, y_df):
    xytrain = pd.concat([x_df, y_df], axis = 1).drop("id", axis = 1)
    bootstrap_sample = xytrain.sample(n = len(xytrain), replace = True)
    bootstrap_x = bootstrap_sample.drop('y', axis = 1)
    bootstrap_y = bootstrap_sample.loc[:,'y']
    bootstrap_y = pd.DataFrame(data = {'y': bootstrap_y.values}, index = bootstrap_y.index)
    return bootstrap_x, bootstrap_y

In [41]:
# Bootstrap pipeline CODE IN PROGRESS
'''

kf = KFold(n_splits=5, shuffle = True)

BMAC_scores = np.array([])
for train_index, test_index in kf.split(xtrain.values):
    
    x_train = xtrain.loc[train_index, :]
    x_test = xtrain.loc[test_index, :]
    
    y_train = ytrain.loc[train_index, :]
    y_test = ytrain.loc[test_index, :]
    
    no_of_bootstraps = 2
    predictions = list()
    for i in np.arange(no_of_bootstraps):
        
        # Generate bootstrap sample
        bootstrap_x, bootstrap_y = generate_bootstrap_sample(x_train, y_train)
        
        # Scale the data
        scaler = preprocessing.StandardScaler()
        xtrain_scaled = scaler.fit_transform(bootstrap_x)
        bootstrap_x = pd.DataFrame(xtrain_scaled, columns = bootstrap_x.columns)
        
        # Fit a support vector classifier
        clf = SVC(gamma=1)
        clf.fit(bootstrap_x, bootstrap_y.values.ravel())
        
        
        # Predict on test fold and append to predictions
        y_pred = clf.predict(x_test)
        predictions = np.append(predictions, y_pred)
    
    # Combine all bootstrap predictions (take max value of count)
    bagging_predictions = max(set(predictions), key=predictions.count)
    
    # Compute BMAC for bagging_predictions
    BMAC = balanced_accuracy_score(y_test, bagging_predictions)
    BMAC_scores = np.append(BMAC_scores, BMAC)
'''       

AttributeError: 'numpy.ndarray' object has no attribute 'count'

In [ ]:
# Bootstrap pipeline with bagging classifier

kf = KFold(n_splits=5, shuffle = True)

c_values = np.array([0.5, 0.8, 1, 1.5, 2, 3, 5, 8, 12])
BMAC_means = np.array([])
BMAC_stds = np.array([])
for c in c_values:
    BMAC_scores = np.array([])
    for train_index, test_index in kf.split(xtrain.values):

        x_train = xtrain.loc[train_index, :]
        x_test = xtrain.loc[test_index, :]

        y_train = ytrain.loc[train_index, :]
        y_test = ytrain.loc[test_index, :]

        # Handle class imbalance with smote function
        sm = SMOTE(random_state=42)
        x_train, y_train = sm.fit_resample(x_train.values, y_train.values) #x_train and y_train are now arrays

        # Scale the data (should this be done for each bootstrap sample? in that case how)
        scaler = preprocessing.StandardScaler()
        x_train = scaler.fit_transform(x_train)
        #x_train = pd.DataFrame(xtrain_scaled, columns = x_train.columns)
        x_test = scaler.fit_transform(x_test)
        #x_test = pd.DataFrame(xtest_scaled, columns = x_test.columns)

        # Model to fit
        estimator = SVC(C=c)
        #estimator = xgb.XGBClassifier(objective="multi:softprob", random_state=40)
        '''
        bagging = BaggingClassifier(base_estimator = estimator,
                                   n_estimators = 20)
        bagging.fit(x_train, y_train)
        bagging_prediction = bagging.predict(x_test)
        BMAC = balanced_accuracy_score(y_test, bagging_prediction)
        '''
        estimator.fit(x_train, y_train.ravel())
        pred = estimator.predict(x_test)
        BMAC = balanced_accuracy_score(y_test, pred)

        BMAC_scores = np.append(BMAC_scores, BMAC)

    BMAC_means = np.append(BMAC_means, np.mean(BMAC_scores))
    BMAC_stds = np.append(BMAC_stds, np.std(BMAC_scores))
    #print(BMAC_scores)
    #print(np.mean(BMAC_scores))
    #print(np.std(BMAC_scores))

print(BMAC_means)
print(BMAC_stds)

/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

In [4]:
# Pipeline on all training data and export to csv

# Handle class imbalance with smote function
sm = SMOTE(random_state=42)
xtrain, ytrain = sm.fit_resample(xtrain.values, ytrain.values) #x_train and y_train are now arrays

# Scale the data (should this be done for each bootstrap sample? in that case how)
scaler = preprocessing.StandardScaler()
xtrain = scaler.fit_transform(xtrain)
#x_train = pd.DataFrame(xtrain_scaled, columns = x_train.columns)
xtest = scaler.fit_transform(xtest)
#x_test = pd.DataFrame(xtest_scaled, columns = x_test.columns)

# Model to fit
estimator = SVC()
#estimator = xgb.XGBClassifier(objective="multi:softprob", random_state=40)
'''
bagging = BaggingClassifier(base_estimator = estimator,
n_estimators = 50)
bagging.fit(x_train, y_train)
bagging_prediction = bagging.predict(x_test)
BMAC = balanced_accuracy_score(y_test, bagging_prediction)
'''
estimator.fit(xtrain, ytrain)
y_pred = estimator.predict(xtest)

index = pd.read_csv("sample.csv")
index['y'] = y_pred

index.to_csv("predictionsSMOTESVM.csv")

/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [5]:
print(np.mean(y_pred), np.std(y_pred))

1.0263414634146342 0.6631696104189518
